In [10]:
 """
        @Author: Alexander Pabel
"""
from PIL import Image
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

#Durch den ResidualBlock wird das Problem der verschwindenden Gradienten behoben, was das Erlernen komplexerer Features ermoeglicht
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    #Dies geschieht ueber einen Shortcut
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
        
class CatDogClassifierResNet(nn.Module):
    def __init__(self):
        super(CatDogResNet, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.layer1 = self._make_layer(32, 64, stride=2)
        self.layer2 = self._make_layer(64, 128, stride=2)
        self.layer3 = self._make_layer(128, 256, stride=2)
        
        self.fc = nn.Sequential(
            nn.Linear(256 * 4 * 4, 512),  # Match final feature map size
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.Linear(512, 2)
        )

    def _make_layer(self, in_channels, out_channels, stride):
        return nn.Sequential(
            ResidualBlock(in_channels, out_channels, stride),
            ResidualBlock(out_channels, out_channels)
        )
    
    def forward(self, x):
        x = self.initial(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = F.avg_pool2d(x, 4)  # Global Average Pooling
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
# Load the model
model = CatDogClassifierResNet()
model.load_state_dict(torch.load('pretrained_model_backup/model_Resnet.pth',weights_only=True),strict=True)
model.eval()

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Bild laden
image_path = './dataset/val/dogs/dog_4026.jpg'
image = Image.open(image_path).convert('RGB')

# Anwenden der transformationen
image = transform(image)

# Stapel dimension angeben
image = image.unsqueeze(0)  # Füge eine Stapeldimension hinzu

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
image = image.to(device)

# Modelvorhersagen bekommen
with torch.no_grad():
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)

# Den Index der Vorhersage auf das Klassenlabel mappen
idx_to_class = {0: 'cats', 1: 'dogs'}
predicted_class = idx_to_class[predicted.item()]
print(f'Predicted class: {predicted_class}')